# PRACTISE DUCKDB

## SETUP & CONFIGURATION

In [45]:
import duckdb

# create duckdb settings dataframe
df = duckdb.sql("select * from duckdb_settings();").to_df()
df.head()

,name,value,description,input_type,scope
0,access_mode,automatic,"Access mode of the database (AUTOMATIC, READ_O...",VARCHAR,GLOBAL
1,allow_persistent_secrets,true,"Allow the creation of persistent secrets, that...",BOOLEAN,GLOBAL
2,checkpoint_threshold,16.0 MiB,The WAL size threshold at which to automatical...,VARCHAR,GLOBAL
3,debug_checkpoint_abort,none,DEBUG SETTING: trigger an abort while checkpoi...,VARCHAR,GLOBAL
4,storage_compatibility_version,v0.10.2,Serialize on checkpoint with compatibility for...,VARCHAR,GLOBAL


## QUICK START

In [2]:
import duckdb

connection = duckdb.connect()

In [6]:
# check how the output is shown
duckdb.sql("SELECT 42*8")

┌──────────┐
│ (42 * 8) │
│  int32   │
├──────────┤
│      336 │
└──────────┘

In [11]:
results = duckdb.sql("SELECT 42").fetchall()
print(results[0][0])
print(results)

42
[(42,)]


In [15]:
df = duckdb.sql("select 42").df()
df

,42
0,42


## PRACTISE

In [2]:
import duckdb
import pandas as pd

df = pd.read_csv("../data/currency_rate_local.csv")

In [4]:
df.head()

,timestamp,currency_code,exchange_rate
0,1690156800,EUR,0.914923
1,1690156800,GBP,0.782291
2,1690156800,INR,82.109338
3,1690156800,AUD,1.471667
4,1690156800,CAD,1.321980


In [8]:
duckdb.filter(df, "currency_code='EUR'")

┌────────────┬───────────────┬───────────────┐
│ timestamp  │ currency_code │ exchange_rate │
│   int64    │    varchar    │    double     │
├────────────┼───────────────┼───────────────┤
│ 1690156800 │ EUR           │      0.914923 │
└────────────┴───────────────┴───────────────┘

## PYTHON API

[python api for duckdb](https://duckdb.org/docs/api/python/overview)

### IN MEMORY STORAGE

In [13]:
# use this for in memory (data is only for that session)
import duckdb

con = duckdb.connect(":memory:")
con.sql("SELECT * FROM '../data/inventory.csv'")

┌─────────────────┬──────────────────────┬──────────────────────┬───┬────────────┬──────────────┬──────────────────────┐
│   ref_number    │       title_en       │       title_fr       │ … │ user_votes │  owner_org   │   owner_org_title    │
│     varchar     │       varchar        │       varchar        │   │   int64    │   varchar    │       varchar        │
├─────────────────┼──────────────────────┼──────────────────────┼───┼────────────┼──────────────┼──────────────────────┤
│ OD-2017-00001   │ Proactive Disclosu…  │ Divulgation proact…  │ … │        372 │ atssc-scdata │ Administrative Tri…  │
│ OD-2017-00002   │ Proactive Disclosu…  │ Divulgation proact…  │ … │        319 │ atssc-scdata │ Administrative Tri…  │
│ OD-2017-00003   │ Proactive Disclosu…  │ Divulgation proact…  │ … │        366 │ atssc-scdata │ Administrative Tri…  │
│ OD-2017-00004   │ Disclosure of Posi…  │ Divulgation des re…  │ … │        351 │ atssc-scdata │ Administrative Tri…  │
│ OD-2017-00005   │ Completed Ac

In [25]:
# in-memory storage
import duckdb

con = duckdb.connect()
con.sql("SELECT * FROM '../data/inventory.csv' LIMIT 5")

┌───────────────┬──────────────────────┬──────────────────────┬───┬────────────┬──────────────┬──────────────────────┐
│  ref_number   │       title_en       │       title_fr       │ … │ user_votes │  owner_org   │   owner_org_title    │
│    varchar    │       varchar        │       varchar        │   │   int64    │   varchar    │       varchar        │
├───────────────┼──────────────────────┼──────────────────────┼───┼────────────┼──────────────┼──────────────────────┤
│ OD-2017-00001 │ Proactive Disclosu…  │ Divulgation proact…  │ … │        372 │ atssc-scdata │ Administrative Tri…  │
│ OD-2017-00002 │ Proactive Disclosu…  │ Divulgation proact…  │ … │        319 │ atssc-scdata │ Administrative Tri…  │
│ OD-2017-00003 │ Proactive Disclosu…  │ Divulgation proact…  │ … │        366 │ atssc-scdata │ Administrative Tri…  │
│ OD-2017-00004 │ Disclosure of Posi…  │ Divulgation des re…  │ … │        351 │ atssc-scdata │ Administrative Tri…  │
│ OD-2017-00005 │ Completed Access t…  │ Demande

In [26]:
con.sql("SELECT ref_number FROM '../data/inventory.csv' LIMIT 5")

┌───────────────┐
│  ref_number   │
│    varchar    │
├───────────────┤
│ OD-2017-00001 │
│ OD-2017-00002 │
│ OD-2017-00003 │
│ OD-2017-00004 │
│ OD-2017-00005 │
└───────────────┘

### PERSISTANT STORAGE

In [14]:
# create a database
import duckdb

con = duckdb.connect("../data/inventory.db")
con.sql("SELECT * FROM '../data/inventory.csv'")
con.close()

In [15]:
config = {"access_mode": "READ_ONLY"}

In [21]:
# load data into the database

# Establishing connection
con = duckdb.connect(
    "../data/inventory.db",
    # config=config,
)

con.sql("CREATE TABLE tbl_inventory AS SELECT * FROM '../data/inventory.csv'")

In [22]:
con.close()

### POLARS

In [68]:
import polars as pl

dl = pl.read_csv("../data/inventory.csv")
dl.head()

ref_number,title_en,title_fr,description_en,description_fr,publisher_en,publisher_fr,date_published,language,size,eligible_for_release,program_alignment_architecture_en,program_alignment_architecture_fr,date_released,portal_url_en,portal_url_fr,user_votes,owner_org,owner_org_title
str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str,str
"""OD-2017-00001""","""Proactive Disclosure of Travel…","""Divulgation proactive des frai…","""This data provides information…","""Ce données fournissent des ren…","""ATSSC""","""SCDATA""","""2014-11-01""","""en+fr""",null,"""Y""","""1.1 Program: Tribunal Speciali…","""Services de soutien spécialisé…",null,null,null,372,"""atssc-scdata""","""Administrative Tribunals Suppo…"
"""OD-2017-00002""","""Proactive Disclosure of Hospit…","""Divulgation proactive des frai…","""This data provides information…","""Ce données fournissent des ren…","""ATSSC""","""SCDATA""","""2014-11-01""","""en+fr""",null,"""Y""","""1.1 Program: Tribunal Speciali…","""Services de soutien spécialisé…",null,null,null,319,"""atssc-scdata""","""Administrative Tribunals Suppo…"
"""OD-2017-00003""","""Proactive Disclosure of Contra…","""Divulgation proactive des marc…","""This data provides information…","""Ces données fournissent des re…","""ATSSC""","""SCDATA""","""2014-11-01""","""en+fr""",null,"""Y""","""1.1 Program: Tribunal Speciali…","""Services de soutien spécialisé…",null,null,null,366,"""atssc-scdata""","""Administrative Tribunals Suppo…"
"""OD-2017-00004""","""Disclosure of Position Reclass…","""Divulgation des reclassificati…","""This data provides information…","""Ces données fournissent des r…","""ATSSC""","""SCDATA""","""2014-11-01""","""en+fr""",null,"""Y""","""1.1 Program: Tribunal Speciali…","""Services de soutien spécialisé…",null,null,null,351,"""atssc-scdata""","""Administrative Tribunals Suppo…"
"""OD-2017-00005""","""Completed Access to Informatio…","""Demandes d’accès à l’informati…","""This data provides information…","""Ces données fournissent des r…","""ATSSC""","""SCDATA""","""2014-11-01""","""en+fr""",null,"""Y""","""1.1 Program: Tribunal Speciali…","""Services de soutien spécialisé…",null,null,null,361,"""atssc-scdata""","""Administrative Tribunals Suppo…"


## CONNECTION OBJECTS